## All Techniques Of Hyper Parameter Optimization
    1.GridSearchCV
    2.RandomizedSearchCV
    3.Bayesian Optimization -Automate Hyperparameter Tuning (Hyperopt)
    4.Sequential Model Based Optimization(Tuning a scikit-learn estimator with skopt)
    5.Optuna- Automate Hyperparameter Tuning
    6.Genetic Algorithms (TPOT Classifier)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
df=pd.read_csv('feature_Selection.csv')
df.head()

,PriorDefault_t,YearsEmployed,CreditScore,Income,Approved
0,1,1.25,1,321.692754,1
1,1,3.04,6,560.000000,1
2,1,1.50,0,824.000000,1
3,1,3.75,5,3.000000,1
4,1,1.71,0,321.692754,1


In [3]:
#### Independent And Dependent features
X=df.drop('Approved',axis=1)
y=df['Approved']

In [4]:
#### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

In [5]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier=RandomForestClassifier(n_estimators=10).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)

In [6]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(f'confusion matrix : {confusion_matrix(y_test,prediction)}')
print(f'Accuracy Score : {accuracy_score(y_test,prediction)}')
print(classification_report(y_test,prediction))

confusion matrix : [[67 11]
 [14 46]]
Accuracy Score : 0.8188405797101449
              precision    recall  f1-score   support

           0       0.83      0.86      0.84        78
           1       0.81      0.77      0.79        60

    accuracy                           0.82       138
   macro avg       0.82      0.81      0.81       138
weighted avg       0.82      0.82      0.82       138



# Manual Hyperparameter Tuning

In [7]:
### Manual Hyperparameter Tuning
model=RandomForestClassifier(n_estimators=300,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=100).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

[[67 11]
 [10 50]]
0.8478260869565217
              precision    recall  f1-score   support

           0       0.87      0.86      0.86        78
           1       0.82      0.83      0.83        60

    accuracy                           0.85       138
   macro avg       0.84      0.85      0.85       138
weighted avg       0.85      0.85      0.85       138



# Randomized Search Cv

In [8]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [9]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [10]:
rf_randomcv.best_params_

{'n_estimators': 1400,
 'min_samples_split': 10,
 'min_samples_leaf': 8,
 'max_features': 'auto',
 'max_depth': 450,
 'criterion': 'gini'}

In [11]:
rf_randomcv

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [12]:
best_random_grid=rf_randomcv.best_estimator_

In [13]:
from sklearn.metrics import accuracy_score
y_pred=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))

[[69  9]
 [10 50]]
Accuracy Score 0.8623188405797102
Classification report:               precision    recall  f1-score   support

           0       0.87      0.88      0.88        78
           1       0.85      0.83      0.84        60

    accuracy                           0.86       138
   macro avg       0.86      0.86      0.86       138
weighted avg       0.86      0.86      0.86       138



# GridSearch CV

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf']+4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split']-2,
                          rf_randomcv.best_params_['min_samples_split']-1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split']+1,
                          rf_randomcv.best_params_['min_samples_split']+2],
    'n_estimators': [rf_randomcv.best_params_['n_estimators']-200, rf_randomcv.best_params_['n_estimators']-100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators']+100, rf_randomcv.best_params_['n_estimators']+200]
}
print(param_grid)

{'criterion': ['gini'], 'max_depth': [450], 'max_features': ['auto'], 'min_samples_leaf': [8, 10, 12], 'min_samples_split': [8, 9, 10, 11, 12], 'n_estimators': [1200, 1300, 1400, 1500, 1600]}


In [15]:
#### Fit the grid_search to the data
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 75 candidates, totalling 750 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [450],
                         'max_features': ['auto'],
                         'min_samples_leaf': [8, 10, 12],
                         'min_samples_split': [8, 9, 10, 11, 12],
                         'n_estimators': [1200, 1300, 1400, 1500, 1600]},
             verbose=2)

In [16]:
grid_search.best_estimator_

RandomForestClassifier(max_depth=450, min_samples_leaf=10, min_samples_split=9,
                       n_estimators=1200)

In [17]:
best_grid=grid_search.best_estimator_

In [18]:
best_grid

RandomForestClassifier(max_depth=450, min_samples_leaf=10, min_samples_split=9,
                       n_estimators=1200)

In [19]:
y_pred=best_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))

[[68 10]
 [10 50]]
Accuracy Score 0.855072463768116
Classification report:               precision    recall  f1-score   support

           0       0.87      0.87      0.87        78
           1       0.83      0.83      0.83        60

    accuracy                           0.86       138
   macro avg       0.85      0.85      0.85       138
weighted avg       0.86      0.86      0.86       138



### Automated Hyperparameter Tuning
Automated Hyperparameter Tuning can be done by using techniques such as

- Bayesian Optimization
- Gradient Descent
- Evolutionary Algorithms
    
#### Bayesian Optimization
Bayesian optimization uses probability to find the minimum of a function. The final aim is to find the input value to a function which can gives us the lowest possible output value.It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

- Objective Function = defines the loss function to minimize.
- Domain Space = defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).
- Optimization Algorithm = defines the search algorithm to use to select the best input values to use in each new iteration.

In [20]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [21]:
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.choice('max_depth',[10, 120, 230, 340, 450, 560, 670, 780, 890, 1000]),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000])
    }

In [22]:
def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [23]:
from sklearn.model_selection import cross_val_score
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 80,
            trials= trials)
best

100%|███████████████████████████████████████████████| 80/80 [08:56<00:00,  6.71s/trial, best loss: -0.8585913185913185]


{'criterion': 1,
 'max_depth': 0,
 'max_features': 1,
 'min_samples_leaf': 0.08453684676276399,
 'min_samples_split': 0.11006204582812613,
 'n_estimators': 2}

In [24]:
crit = {0: 'entropy', 1: 'gini'}
depth = [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000]
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 200, 1: 400, 2: 600, 3: 800, 4: 1000, 5:1200, 6:1400,7:1600,8:1800,9:2000}

print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

gini
sqrt
600


In [25]:
best['min_samples_leaf']

0.08453684676276399

In [26]:
trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = depth[best['max_depth']], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]).fit(X_train,y_train)
predictionforest = trainedforest.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(accuracy_score(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc5 = accuracy_score(y_test,predictionforest)

[[64 14]
 [ 6 54]]
0.855072463768116
              precision    recall  f1-score   support

           0       0.91      0.82      0.86        78
           1       0.79      0.90      0.84        60

    accuracy                           0.86       138
   macro avg       0.85      0.86      0.85       138
weighted avg       0.86      0.86      0.86       138



## Genetic Algorithms
Genetic Algorithms tries to apply natural selection mechanisms to Machine Learning contexts.

Let's imagine we create a population of N Machine Learning models with some predifined Hyperparameters. We can then calculate the accuracy of each model and decide to keep just half of the models (the ones that performs best). We can now generate some offsprings having similar Hyperparameters to the ones of the best models so that go get again a population of N models. At this point we can again caltulate the accuracy of each model and repeate the cycle for a defined number of generations. In this way, just the best models will survive at the end of the process.

In [27]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [28]:
from tpot import TPOTClassifier

tpot_classifier = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier': param}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(X_train,y_train)

Optimization Progress:   0%|          | 0/84 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8586956521739131

Generation 2 - Current best internal CV score: 0.8586956521739131

Generation 3 - Current best internal CV score: 0.8586956521739131

Generation 4 - Current best internal CV score: 0.8586956521739131

Generation 5 - Current best internal CV score: 0.8586956521739131

Best pipeline: RandomForestClassifier(input_matrix, criterion=gini, max_depth=120, max_features=auto, min_samples_leaf=8, min_samples_split=5, n_estimators=1400)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                 

In [29]:
accuracy = tpot_classifier.score(X_test, y_test)
print(accuracy)

0.8478260869565217


## Optimize hyperparameters of the model using Optuna
The hyperparameters of the above algorithm are n_estimators and max_depth for which we can try different values to see if the model accuracy can be improved. The objective function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [30]:
import optuna
import sklearn.svm
def objective(trial):
    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        clf = sklearn.svm.SVC(C=c, gamma='auto')
    return sklearn.model_selection.cross_val_score(
        clf,X_train,y_train, n_jobs=-1, cv=3).mean()

In [31]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-11-07 09:21:53,119] A new study created in memory with name: no-name-d1f2e607-527d-4c27-ad91-968e601eb6d4
[I 2021-11-07 09:21:58,012] Trial 0 finished with value: 0.8278985507246377 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1860, 'max_depth': 52.831636492821204}. Best is trial 0 with value: 0.8278985507246377.
[I 2021-11-07 09:22:03,753] Trial 1 finished with value: 0.6721014492753623 and parameters: {'classifier': 'SVC', 'svc_c': 274357.6778465932}. Best is trial 0 with value: 0.8278985507246377.
[I 2021-11-07 09:22:06,332] Trial 2 finished with value: 0.8333333333333334 and parameters: {'classifier': 'RandomForest', 'n_estimators': 700, 'max_depth': 66.28649973761492}. Best is trial 2 with value: 0.8333333333333334.
[I 2021-11-07 09:22:07,029] Trial 3 finished with value: 0.8278985507246377 and parameters: {'classifier': 'RandomForest', 'n_estimators': 420, 'max_depth': 21.09713286033926}. Best is trial 2 with value: 0.8333333333333334.
[I 2021-11-07 09:2

Accuracy: 0.8333333333333334
Best hyperparameters: {'classifier': 'RandomForest', 'n_estimators': 700, 'max_depth': 66.28649973761492}


In [32]:
trial

FrozenTrial(number=2, values=[0.8333333333333334], datetime_start=datetime.datetime(2021, 11, 7, 9, 22, 3, 754986), datetime_complete=datetime.datetime(2021, 11, 7, 9, 22, 6, 331670), params={'classifier': 'RandomForest', 'n_estimators': 700, 'max_depth': 66.28649973761492}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest', 'SVC')), 'n_estimators': IntUniformDistribution(high=2000, low=200, step=10), 'max_depth': LogUniformDistribution(high=100.0, low=10.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None)

In [33]:
study.best_params

{'classifier': 'RandomForest',
 'n_estimators': 700,
 'max_depth': 66.28649973761492}

In [38]:
rf=RandomForestClassifier(n_estimators=study.best_params['n_estimators'],max_depth=study.best_params['max_depth'])
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=66.28649973761492, n_estimators=700)

In [39]:
y_pred=rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[66 12]
 [14 46]]
0.8115942028985508
              precision    recall  f1-score   support

           0       0.82      0.85      0.84        78
           1       0.79      0.77      0.78        60

    accuracy                           0.81       138
   macro avg       0.81      0.81      0.81       138
weighted avg       0.81      0.81      0.81       138



In [40]:
import pickle
# save the model to disk
filename = 'finalized_model_RF.sav'
pickle.dump(rf_randomcv, open(filename, 'wb'))